                                             Stock Analysis 
                                            

In [1]:
# Dependancies
import pandas as pd
import os
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from sqlalchemy import func
import warnings
warnings.filterwarnings('ignore')
from config import db_pass
import pymysql
from functools import reduce
from sqlalchemy import Column, Integer, String, Float
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'config'

In [2]:
# Establish the connection 
rds_connection_string = f"root:Qweasdzxc123!@127.0.0.1/stocks_db"
engine = create_engine(f'mysql+pymysql://{rds_connection_string}')
# conn = engine.connect()

                                                Part I
                                             Data Minging
                         Cleaning the data, merging two dataframes for further analysis

In [3]:
# Read in nyse data from the table
nyse_raw_df = pd.read_sql('SELECT * FROM companylist_nyse', con=engine)
nyse_raw_df

,id,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,SummaryQuote
0,1,DDD,3D Systems Corporation,10.7500,1250.00,None,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/ddd
1,2,MMM,3M Company,190.2100,109640.00,None,Health Care,Medical/Dental Instruments,https://www.nasdaq.com/symbol/mmm
2,3,WBAI,500.com Limited,11.9700,508.97,2013,Consumer Services,Services-Misc. Amusement & Recreation,https://www.nasdaq.com/symbol/wbai
3,4,WUBA,58.com Inc.,71.6900,10630.00,2013,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/wuba
4,5,EGHT,8x8 Inc,23.7000,2270.00,None,Technology,EDP Services,https://www.nasdaq.com/symbol/eght
5,6,AHC,A.H. Belo Corporation,3.7400,80.59,None,Consumer Services,Newspapers/Magazines,https://www.nasdaq.com/symbol/ahc
6,7,AOS,A.O Smith Corporation,55.8100,9370.00,None,Consumer Durables,Consumer Electronics/Appliances,https://www.nasdaq.com/symbol/aos
7,8,ATEN,"A10 Networks, Inc.",6.5800,494.71,2014,Technology,Computer Communications Equipment,https://www.nasdaq.com/symbol/aten
8,9,AAC,"AAC Holdings, Inc.",1.7200,42.45,2014,Health Care,Medical Specialities,https://www.nasdaq.com/symbol/aac
9,10,AIR,AAR Corp.,33.8100,1190.00,None,Capital Goods,Aerospace,https://www.nasdaq.com/symbol/air


In [4]:
# Drop unnecessary columns in nyse df
nyse_df = nyse_raw_df.drop(columns=['id', 'Name', 'LastSale', 'IPOyear', 'SummaryQuote'])
nyse_df.shape

(3112, 4)

In [5]:
# Drop Nan Values in nyse df
nyse_df = nyse_df.dropna()
nyse_df.shape

(1964, 4)

In [6]:
# Read in nasdaq data from the table
nasdaq_raw_df = pd.read_sql('SELECT * FROM cleandf_ndaq', con=engine)
nasdaq_raw_df

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,industry,Summary Quote
0,YI,"111, Inc.",8.6500,705.36,2018.0,Health Care,Medical/Nursing Services,https://www.nasdaq.com/symbol/yi
1,PIH,"1347 Property Insurance Holdings, Inc.",5.0650,30.45,2014.0,Finance,Property-Casualty Insurers,https://www.nasdaq.com/symbol/pih
2,FLWS,"1-800 FLOWERS.COM, Inc.",19.4100,1.25,1999.0,Consumer Services,Other Specialty Stores,https://www.nasdaq.com/symbol/flws
3,BCOW,"1895 Bancorp of Wisconsin, Inc.",9.6500,47.06,2019.0,Finance,Banks,https://www.nasdaq.com/symbol/bcow
4,VNET,"21Vianet Group, Inc.",8.0500,904.76,2011.0,Technology,"Computer Software: Programming, Data Processing",https://www.nasdaq.com/symbol/vnet
5,TWOU,"2U, Inc.",61.6100,3.60,2014.0,Technology,Computer Software: Prepackaged Software,https://www.nasdaq.com/symbol/twou
6,QFIN,"360 Finance, Inc.",20.0200,2.88,2018.0,Finance,Finance: Consumer Services,https://www.nasdaq.com/symbol/qfin
7,JOBS,"51job, Inc.",93.8700,5.81,2004.0,Technology,Diversified Commercial Services,https://www.nasdaq.com/symbol/jobs
8,JFKKU,8i Enterprises Acquisition Corp,10.0700,50.35,2019.0,Finance,Business Services,https://www.nasdaq.com/symbol/jfkku
9,ABIL,Ability Inc.,1.4200,11.42,2014.0,Capital Goods,Telecommunications Equipment,https://www.nasdaq.com/symbol/abil


In [7]:
# Drop unnecessary columns
nasdaq_df = nasdaq_raw_df.drop(columns=['Symbol', 'Name', 'LastSale', 'IPOyear', 'Summary Quote'])

In [8]:
# Concat both data frames
# Save dataframes into list
frames = [nyse_df, nasdaq_df]
# Concat 
result_df = pd.concat(frames)
result_df.head(2)

,MarketCap,Sector,Symbol,industry
0,1250.0,Technology,DDD,Computer Software: Prepackaged Software
1,109640.0,Health Care,MMM,Medical/Dental Instruments


In [9]:
# Check duplicates for tickers 
result_df['Symbol'].value_counts()

HCP     1
KL      1
CRS     1
PVL     1
HLT     1
AKR     1
RMAX    1
SSI     1
FDC     1
RY      1
SRC     1
HIL     1
BSIG    1
NBLX    1
CEQP    1
MFG     1
OZM     1
AI      1
KEN     1
VRTV    1
PLD     1
RIO     1
BAC     1
JCAP    1
NAO     1
TRGP    1
KOS     1
BBY     1
HEP     1
CNO     1
       ..
MPLX    1
MBT     1
LPG     1
OI      1
AVX     1
FUN     1
BEDU    1
VEC     1
FSM     1
EQM     1
MPX     1
BHR     1
DVA     1
SAFE    1
SDRL    1
MTOR    1
HMLP    1
PZN     1
RST     1
CVNA    1
CIT     1
EGO     1
MN      1
ZTS     1
DEO     1
ALL     1
MCS     1
NLSN    1
CS      1
JMIA    1
Name: Symbol, Length: 1964, dtype: int64

In [10]:
# Check if the values are the same for each duplicate 
result_df[result_df['Symbol']=='PGTI']

,MarketCap,Sector,Symbol,industry
2194,860.79,Capital Goods,PGTI,Building Products


In [13]:
# Dropping the symbol's duplicates
result_stock_df = result_df.drop_duplicates()
result_stock_df

,MarketCap,Sector,Symbol,industry
0,1250.00,Technology,DDD,Computer Software: Prepackaged Software
1,109640.00,Health Care,MMM,Medical/Dental Instruments
2,508.97,Consumer Services,WBAI,Services-Misc. Amusement & Recreation
3,10630.00,Technology,WUBA,"Computer Software: Programming, Data Processing"
4,2270.00,Technology,EGHT,EDP Services
5,80.59,Consumer Services,AHC,Newspapers/Magazines
6,9370.00,Consumer Durables,AOS,Consumer Electronics/Appliances
7,494.71,Technology,ATEN,Computer Communications Equipment
8,42.45,Health Care,AAC,Medical Specialities
9,1190.00,Capital Goods,AIR,Aerospace


In [19]:
result_stock_df = result_stock_df.dropna(subset=['Symbol'])
result_stock_df

,MarketCap,Sector,Symbol,industry
0,1250.00,Technology,DDD,Computer Software: Prepackaged Software
1,109640.00,Health Care,MMM,Medical/Dental Instruments
2,508.97,Consumer Services,WBAI,Services-Misc. Amusement & Recreation
3,10630.00,Technology,WUBA,"Computer Software: Programming, Data Processing"
4,2270.00,Technology,EGHT,EDP Services
5,80.59,Consumer Services,AHC,Newspapers/Magazines
6,9370.00,Consumer Durables,AOS,Consumer Electronics/Appliances
7,494.71,Technology,ATEN,Computer Communications Equipment
8,42.45,Health Care,AAC,Medical Specialities
9,1190.00,Capital Goods,AIR,Aerospace


                                              PART II
                                Uploading a new table into DataBase

In [20]:
# Transporting the dataframe to the 
result_stock_df.to_sql(name='stock_final_analysis', con=engine, if_exists='append', index=False)

In [21]:
# Check to see if the data was transformed 
pd.read_sql_query('select * from stock_final_analysis', con=engine).head()

,id,Symbol,MarketCap,Sector,industry
0,3354,DDD,1250.00,Technology,Computer Software: Prepackaged Software
1,3355,MMM,109640.00,Health Care,Medical/Dental Instruments
2,3356,WBAI,508.97,Consumer Services,Services-Misc. Amusement & Recreation
3,3357,WUBA,10630.00,Technology,"Computer Software: Programming, Data Processing"
4,3358,EGHT,2270.00,Technology,EDP Services


In [22]:
# Analysing sectors in analysis table
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)
# Check the DataBase for tables 
Base.classes.keys()

['all_stocks_5yr',
 'companylist_nasdq',
 'companylist_nyse',
 'stock_final_analysis']

In [23]:
# Save references to analysis table
Stock_final_analysis = Base.classes.stock_final_analysis

# Create a session
session = Session(engine)

# Create the inspector and connect it to the engine
inspector = inspect(engine)

# inspecting the columns
columns = inspector.get_columns('stock_final_analysis')   
for column in columns:
    print(column["name"], column["type"])

id INTEGER(11)
Symbol VARCHAR(30)
MarketCap DECIMAL(18, 2)
Sector VARCHAR(50)
industry VARCHAR(100)


                                             PART III
                                    Sector & Industry Analysis

In [24]:
# Filter by Sector to reveal which sector has the most shares 
sector = session.query(Stock_final_analysis.Sector, func.count(Stock_final_analysis.Symbol)).\
    group_by(Stock_final_analysis.Sector).\
    order_by(func.count(Stock_final_analysis.Symbol).desc()).all()
print(sector)

[('Consumer Services', 420), ('Finance', 283), ('Energy', 199), ('Basic Industries', 186), ('Capital Goods', 183), ('Technology', 173), ('Public Utilities', 141), ('Consumer Non-Durables', 106), ('Health Care', 104), ('Consumer Durables', 60), ('Transportation', 59), ('Miscellaneous', 50)]


In [25]:
# How much money is in the Each Sector
fin = session.query(Stock_final_analysis.Sector, func.sum(Stock_final_analysis.MarketCap)).\
    group_by(Stock_final_analysis.Sector).\
    order_by(func.count(Stock_final_analysis.Symbol).desc()).all()
print(fin)

[('Consumer Services', Decimal('3504047.65')), ('Finance', Decimal('5242123.79')), ('Energy', Decimal('2808570.13')), ('Basic Industries', Decimal('2091738.14')), ('Capital Goods', Decimal('2344332.03')), ('Technology', Decimal('2106031.83')), ('Public Utilities', Decimal('2678465.48')), ('Consumer Non-Durables', Decimal('2095315.15')), ('Health Care', Decimal('3394360.33')), ('Consumer Durables', Decimal('373675.70')), ('Transportation', Decimal('907877.29')), ('Miscellaneous', Decimal('1560792.40'))]


In [26]:
# Deeper Anaysis by Industry
industry = session.query(Stock_final_analysis.industry, func.count(Stock_final_analysis.industry)).\
    filter(Stock_final_analysis.Sector == 'Consumer Services').\
    group_by(Stock_final_analysis.industry).\
    order_by(func.count(Stock_final_analysis.industry).desc()).all()
print(industry)

[('Real Estate Investment Trusts', 172), ('Other Consumer Services', 35), ('Clothing/Shoe/Accessory Stores', 20), ('Marine Transportation', 19), ('Hotels/Resorts', 19), ('Other Specialty Stores', 16), ('Restaurants', 15), ('Services-Misc. Amusement & Recreation', 12), ('Department/Specialty Retail Stores', 11), ('Broadcasting', 11), ('RETAIL: Building Materials', 9), ('Newspapers/Magazines', 7), ('Television Services', 7), ('Catalog/Specialty Distribution', 6), ('Professional Services', 6), ('Transportation Services', 5), ('Movies/Entertainment', 5), ('Food Chains', 5), ('Home Furnishings', 4), ('Military/Government/Technical', 4), ('Diversified Commercial Services', 3), ('Building operators', 3), ('Telecommunications Equipment', 3), ('Office Equipment/Supplies/Services', 2), ('Publishing', 2), ('Advertising', 2), ('Real Estate', 2), ('Consumer Specialties', 2), ('Farming/Seeds/Milling', 2), ('Homebuilding', 2), ('Rental/Leasing Companies', 2), ('Recreational Products/Toys', 1), ('Pape